In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import torch

In [4]:
model = AutoModelForCausalLM.from_pretrained('meta-llama/Meta-Llama-3.1-8B-Instruct', torch_dtype='bfloat16', device_map='auto').eval()

tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3.1-8B-Instruct')

Loading checkpoint shards: 100%|██████████| 4/4 [00:29<00:00,  7.32s/it]


In [25]:
prompt="""
Who was the first President of the United States?
A) Abraham Lincoln
B) George Washington
C) Thomas Jefferson
D) John Adams
The answer is: 
"""

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load pre-trained LLaMA model and tokenizer
# model_name = 'huggingface/llama-base'  # Example model name
# model_name = 'meta-llama/Meta-Llama-3-8B'
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# Ensure model is in evaluation mode
model.eval()

# Input text
# input_text = "Hello, how are you?"
# input_text="""
# Who was the first President of the United States?
# A) Abraham Lincoln
# B) George Washington
# C) Thomas Jefferson
# D) John Adams
# The answer is: B) George Washington
# """

all_inputs = []
input_text='An integer c is a common divisor of two integers x and y if and only if c is a divisor of x and c is a divisor of y. Which of the following sets of integers could possibly be the set of all common divisors of two integers?\nA. {-6,-2, -1, 1, 2, 6}\nB. {-6, -2, -1, 0, 1, 2, 6}\nC. {-6, -3, -2, -1, 1, 2, 3, 6}\nD. {-6, -3, -2, -1, 0, 1, 2, 3, 6}\nAnswer: '

input_text = tokenizer.apply_chat_template(
    [{'role': 'user', 'content': f"Please answer the following question with only A, B, C, D: {input_text}"}],
    add_generation_prompt=True,
    tokenize=False
)

all_inputs = [input_text ]

input_text='Who was the first President of the United States?\nA. Abraham Lincoln\nB. George Washington\nC. Thomas Jefferson\nD. John Adams\nAnswer: '

# input_text += 'D'
input_text = tokenizer.apply_chat_template(
    [
     {'role': 'user', 'content': f"Please answer the question with only A, B, C, D: {input_text}"}],
    add_generation_prompt=True,
    tokenize=False
)

all_inputs.append(input_text)
# input_text="""
# Multiple Choice Question: Who was the first President of the United States? 
# A) Abraham Lincoln
# B) George Washington
# C) Thomas Jefferson
# D) John Adams
# Answer: George Washington
# """
# Tokenize input
tokenizer.padding_side='left'
tokenizer.pad_token = tokenizer.eos_token

# print(input_text)
inputs = tokenizer(input_text, return_tensors="pt", padding=True, ).to(model.device)

# Forward pass to get logits
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Shift logits and labels for language modeling task
shifted_logits = logits[..., :-1, :].contiguous()
shifted_labels = inputs['input_ids'][..., 1:].contiguous()

# Calculate log-probs
log_probs = torch.nn.functional.log_softmax(shifted_logits, dim=-1)

# Get the log-probs of the correct next token
target_log_probs = log_probs.gather(-1, shifted_labels.unsqueeze(-1)).squeeze(-1)

# Convert back to text for inspection
decoded_tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].int())

# Print out log-probs for each token
for token, log_prob in zip(decoded_tokens[1:][-1:], target_log_probs[0][-1:]):
    print(f"Token: {token}, Log-Prob: {log_prob.item()}")

Token: ĊĊ, Log-Prob: 0.0


In [15]:
inputs['input_ids'].shape

torch.Size([1, 82])

In [21]:
logits[..., -1, 426]

tensor([11.3125], device='cuda:0')

In [18]:
tokenizer.decode(inputs['input_ids'][0])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nPlease answer the question with only A, B, C, D: Who was the first President of the United States?\nA. Abraham Lincoln\nB. George Washington\nC. Thomas Jefferson\nD. John Adams\nAnswer:<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'

In [22]:
for c in [' A', ' B', ' C', ' D']:
    print(tokenizer(c)['input_ids'][-1])
    print(logits[..., -1, tokenizer(c)['input_ids'][-1]])

362
tensor([-0.7578], device='cuda:0')
426
tensor([11.3125], device='cuda:0')
356
tensor([0.0767], device='cuda:0')
423
tensor([2.4062], device='cuda:0')


In [23]:
for c in ['A', 'B', 'C', 'D']:
    print(tokenizer(c)['input_ids'][-1])
    print(logits[..., -1, tokenizer(c)['input_ids'][-1]])

32
tensor([25.5000], device='cuda:0')
33
tensor([36.7500], device='cuda:0')
34
tensor([24.5000], device='cuda:0')
35
tensor([26.5000], device='cuda:0')


In [266]:
tokenizer.bos_token_id

128000

In [8]:
torch.nn.functional.softmax(logits[..., -1, :][:, [362, 426, 356, 423]])

/tmp/ipykernel_3807260/3070696669.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.nn.functional.softmax(logits[..., -1, :][:, [362, 426, 356, 423]])


tensor([[5.9546e-06, 9.9991e-01, 7.3243e-06, 8.0927e-05]], device='cuda:0')

In [260]:
import math
log_probs = [
    -0.06564091145992279,
    -5.180295467376709,
    -5.19955587387085,
    -3.9977316856384277
]

for v in log_probs:
    print(math.exp(v)/sum(math.exp(_) for _ in log_probs))

0.9694580571201163
0.0058245553131501095
0.005713445455468459
0.019003942111265108


In [235]:
print(input_text)
print(logits.shape)
print(inputs['input_ids'])
print(tokenizer.decode(inputs['input_ids'][0]))

Who was the first President of the United States?
A. Abraham Lincoln
B. George Washington
C. Thomas Jefferson
D. John Adams
Answer: 
torch.Size([1, 34, 128256])
tensor([[128000,  15546,    574,    279,   1176,   4900,    315,    279,   3723,
           4273,   5380,     32,     13,  37488,  25379,    198,     33,     13,
          10058,   6652,    198,     34,     13,  11355,  34644,    198,     35,
             13,   3842,  27329,    198,  16533,     25,    220]])
<|begin_of_text|>Who was the first President of the United States?
A. Abraham Lincoln
B. George Washington
C. Thomas Jefferson
D. John Adams
Answer: 


In [237]:
print(input_text)
print(logits.shape)
print(inputs['input_ids'])
print(tokenizer.decode(inputs['input_ids'][0]))

Who was the first President of the United States?
A. Abraham Lincoln
B. George Washington
C. Thomas Jefferson
D. John Adams
Answer: B
torch.Size([1, 34, 128256])
tensor([[128000,  15546,    574,    279,   1176,   4900,    315,    279,   3723,
           4273,   5380,     32,     13,  37488,  25379,    198,     33,     13,
          10058,   6652,    198,     34,     13,  11355,  34644,    198,     35,
             13,   3842,  27329,    198,  16533,     25,    426]])
<|begin_of_text|>Who was the first President of the United States?
A. Abraham Lincoln
B. George Washington
C. Thomas Jefferson
D. John Adams
Answer: B


In [229]:
logits.shape

torch.Size([1, 34, 128256])

In [151]:
log_probs = torch.nn.functional.log_softmax(logits, dim=-1)

In [183]:
log_probs[:, -2][..., torch.LongTensor([32, 33, 34, 35])]

tensor([[-12.7291, -11.7851, -11.9531, -10.1250],
        [-15.7022, -10.2309, -15.3585, -13.7300]])

In [153]:
target_log_probs = log_probs.gather(-1, shifted_labels.unsqueeze(-1)).squeeze(-1)


In [277]:
test_df.iloc[0, 5]

'A'

In [272]:
tokenizer('hello world')

{'input_ids': [128000, 15339, 1917], 'attention_mask': [1, 1, 1]}

In [204]:
tokenizer.decode(inputs['input_ids'].tolist()[0])

'<|begin_of_text|>Who was the first President of the United States?\nA. Abraham Lincoln\nB. George Washington\nC. Thomas Jefferson\nD. John Adams\nAnswer: '

In [221]:
logits

torch.Size([1, 34, 128256])

In [82]:
choices = ["A", "B", "C", "D"]
def format_subject(subject):
    l = subject.split("_")
    s = ""
    for entry in l:
        s += " " + entry
    return s

def format_example(df, idx, include_answer=True):
    prompt = df.iloc[idx, 0]
    k = df.shape[1] - 2
    for j in range(k):
        prompt += "\n{}. {}".format(choices[j], df.iloc[idx, j+1])
    prompt += "\nAnswer:"
    if include_answer:
        prompt += " {}\n\n".format(df.iloc[idx, k + 1])
    return prompt

def gen_prompt(train_df, subject, k=-1):
    prompt = "The following are multiple choice questions (with answers) about {}.\n\n".format(format_subject(subject))
    if k == -1:
        k = train_df.shape[0]
    for i in range(k):
        prompt += format_example(train_df, i)
    return prompt


In [83]:
import pandas as pd
test_df = pd.read_csv('college_computer_science_test.csv', header=None)

In [120]:
prompt_end = format_example(test_df, 1, include_answer=True)

In [87]:
label = test_df.iloc[0, test_df.shape[1]-1]

In [271]:
len(test_df)

100

In [121]:
prompt_end

'An integer c is a common divisor of two integers x and y if and only if c is a divisor of x and c is a divisor of y. Which of the following sets of integers could possibly be the set of all common divisors of two integers?\nA. {-6,-2, -1, 1, 2, 6}\nB. {-6, -2, -1, 0, 1, 2, 6}\nC. {-6, -3, -2, -1, 1, 2, 3, 6}\nD. {-6, -3, -2, -1, 0, 1, 2, 3, 6}\nAnswer: C\n\n'

In [40]:
decoded_tokens

['<|begin_of_text|>',
 'Ċ',
 'Who',
 'Ġwas',
 'Ġthe',
 'Ġfirst',
 'ĠPresident',
 'Ġof',
 'Ġthe',
 'ĠUnited',
 'ĠStates',
 '?Ċ',
 'A',
 ')',
 'ĠAbraham',
 'ĠLincoln',
 'Ċ',
 'B',
 ')',
 'ĠGeorge',
 'ĠWashington',
 'Ċ',
 'C',
 ')',
 'ĠThomas',
 'ĠJefferson',
 'Ċ',
 'D',
 ')',
 'ĠJohn',
 'ĠAdams',
 'Ċ',
 'The',
 'Ġanswer',
 'Ġis',
 ':',
 'ĠĊ']

In [9]:
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(model.device)


In [20]:
with torch.no_grad():
    outputs = model.generate(input_ids, return_dict_in_generate=True)
    # Logits for each token in the sequence
    logits = outputs.logits


TypeError: The current model class (LlamaModel) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'LlamaForCausalLM'}

In [21]:
model

LlamaModel(
  (embed_tokens): Embedding(128256, 4096)
  (layers): ModuleList(
    (0-31): 32 x LlamaDecoderLayer(
      (self_attn): LlamaSdpaAttention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (rotary_emb): LlamaRotaryEmbedding()
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm()
      (post_attention_layernorm): LlamaRMSNorm()
    )
  )
  (norm): LlamaRMSNorm()
  (rotary_emb): LlamaRotaryEmbedding()
)

In [1]:
from vllm import SamplingParams

gen_params = SamplingParams(
    temperature=1.0,  # Deterministic output
    top_p=1.0,
    max_tokens=1,  # We don't want to generate new tokens, just evaluate the inputs
    logprobs=True,  # Request log probabilities
)

/data/home/chaoqiw/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


WARNING 10-02 21:02:33 cuda.py:22] You are using a deprecated `pynvml` package. Please install `nvidia-ml-py` instead, and make sure to uninstall `pynvml`. When both of them are installed, `pynvml` will take precedence and cause errors. See https://pypi.org/project/pynvml for more information.


2024-10-02 21:02:46.869361: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-02 21:02:51.556383: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-10-02 21:03:01,012	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


ValueError: max_tokens must be at least 1, got 0.

In [ ]:
import vllm
import os

# You can replace this with the path to a locally saved model or a Hugging Face model
MODEL_NAME = "meta-llama/Meta-Llama-3-8B"  # Example model (replace with appropriate model)

def load_model(model_name):
    """Load the LLaMA model with vllm."""
    # Create a LLaMA model instance
    model = vllm.LLM(model_name)
    return model

def generate_response(model, prompt):
    """Generate a response from the LLaMA model given the input prompt."""
    output = model.generate(prompt, max_tokens=150)
    return output

In [ ]:
model = vllm.LLM(MODEL_NAME, )

In [ ]:


def main():
    # Load the model
    print(f"Loading model '{MODEL_NAME}'...")
    model = load_model(MODEL_NAME)
    print("Model loaded successfully!")

    # Begin the conversation
    print("\n--- Chat with LLaMA ---")
    print("Type 'exit' to end the chat.")

    while True:
        # User input
        prompt = input("\nYou: ")
        if prompt.lower() == 'exit':
            print("Ending chat.")
            break
        
        # Generate response
        print("LLaMA is thinking...")
        response = generate_response(model, prompt)
        print(f"LLaMA: {response}")
